In [1]:
import numpy as np
import pdb
from env.ofdm_env import ISAC_BS

In [7]:
env = ISAC_BS(N=16,N_c=8,N_r=8)
env.U = np.random.binomial(1,0.5,size=(env.N_c,env.N))
env.D = np.random.binomial(1,0.5,size=(env.N_r,env.N))

env.get_performance(print_log=True)
# env.U = np.array([[1,1,1,1,0,0,0,0]])
# env.D = np.array([[0,1,0,0,1,1,1,1]])
# env.get_performance(print_log=True)
next_state,reward,done = env.step()
print("next_state",next_state)
print("reward",reward)

C_SINR(dB):  [3.55695791 3.56552943 3.45266993 3.473787   3.54408736 3.56270839
 3.49814048 3.41984753]
R_SINR(dB):  [-inf -inf -inf -inf -inf -inf -inf -inf]
SUM_R_c:  2907957.9541107705  EE:  45436.84303298079
SUM_MI_r:  0.0  EE:  nan
next_state [0.0039066  0.00390659 0.00390657 0.00390655 0.00390654 0.00390652
 0.0039065  0.00390649 0.00390647 0.00390645 0.00390644 0.00390642
 0.0039064  0.00390639 0.00390637 0.00390635 0.00385317 0.00385315
 0.00385313 0.00385312 0.0038531  0.00385308 0.00385307 0.00385305
 0.00385303 0.00385302 0.003853   0.00385298 0.00385297 0.00385295
 0.00385294 0.00385292 0.00388523 0.00388522 0.0038852  0.00388518
 0.00388517 0.00388515 0.00388513 0.00388512 0.0038851  0.00388508
 0.00388507 0.00388505 0.00388504 0.00388502 0.003885   0.00388499
 0.0031332  0.00313319 0.00313318 0.00313316 0.00313315 0.00313314
 0.00313312 0.00313311 0.0031331  0.00313308 0.00313307 0.00313306
 0.00313304 0.00313303 0.00313302 0.003133   0.00499936 0.00499934
 0.00499932 0.0

/home/dhy/final/ISAC_OFDM/env/ofdm_env.py:121: RuntimeWarning: invalid value encountered in double_scalars
  EE_R = SUM_MI_r/SUM_R_P
/home/dhy/final/ISAC_OFDM/env/ofdm_env.py:124: RuntimeWarning: divide by zero encountered in log10
  print("R_SINR(dB): ",10*np.log10(np.sum(R_SINR,axis=-1)))


In [9]:
np.random.binomial(1,0.5,size=(2,1))

array([[1],
       [0]])